# <center>RPA transferencia entre bancos<center>

## Importações

In [57]:
import psycopg2
import pandas as pd
import os
from dotenv import load_dotenv

## Pegando o login do banco apartir de um arquivo .ENV

In [58]:
load_dotenv()

db_host1 = os.getenv('DB_HOST1')
db_database1 = os.getenv('DB_DATABASE1')
db_user1 = os.getenv('DB_USER1')
db_password1 = os.getenv('DB_PASSWORD1')
db_port1 = os.getenv('DB_PORT1')

db_database2 = os.getenv('DB_DATABASE2')

print("DB_HOST1:", os.getenv('DB_HOST1'))
print("DB_DATABASE1:", os.getenv('DB_DATABASE1'))
print("DB_USER1:", os.getenv('DB_USER1'))
print("DB_PASSWORD1:", os.getenv('DB_PASSWORD1'))
print("DB_PORT1:", os.getenv('DB_PORT1'))
print("DB_DATABASE2:", os.getenv('DB_DATABASE2'))

DB_HOST1: recoop-germinare-9764.h.aivencloud.com
DB_DATABASE1: BANCO_1ANO
DB_USER1: avnadmin
DB_PASSWORD1: AVNS_DAFAJWqxMl1ba9hBbcZ
DB_PORT1: 16983
DB_DATABASE2: defaultdb


## Se conectando no banco do 1ano

In [59]:
print('Começando===============================================')
try:
    # Conectar ao banco de dados
    conn = psycopg2.connect(
        host=db_host1,
        database=db_database1,
        user=db_user1,
        password=db_password1,
        port=db_port1
    )
    print("Conexão estabelecida com sucesso!")
except psycopg2.OperationalError as e:
    print(f"Erro ao conectar ao banco de dados: {e}")
    exit()  # Saia se a conexão falhar

Começando===============================================
Conexão estabelecida com sucesso!


## Pegando as informações do banco de dados

In [60]:
try:
    cursor = conn.cursor()

    cursor.execute('''SELECT * FROM Cooperativa''')
    results = cursor.fetchall()
    
    # Criar DataFrame
    columns = [desc[0] for desc in cursor.description]
    df = pd.DataFrame(results, columns=columns)

    cursor.execute('''SELECT * FROM Cooperativa''')
    columns = [desc[0] for desc in cursor.description]
    results = cursor.fetchall()
    df_cooperativa = pd.DataFrame(results, columns=columns)

    cursor.execute('''SELECT * FROM Endereco''')
    columns = [desc[0] for desc in cursor.description]
    results = cursor.fetchall()
    df_endereco = pd.DataFrame(results, columns=columns)

    cursor.execute('''SELECT * FROM Leilao''')
    columns = [desc[0] for desc in cursor.description]
    results = cursor.fetchall()
    df_leilao = pd.DataFrame(results, columns=columns)

    cursor.execute('''SELECT * FROM Lance''')
    columns = [desc[0] for desc in cursor.description]
    results = cursor.fetchall()
    df_lance = pd.DataFrame(results, columns=columns)

    cursor.execute('''SELECT * FROM Produto''')
    columns = [desc[0] for desc in cursor.description]
    results = cursor.fetchall()
    df_produto = pd.DataFrame(results, columns=columns)

    cursor.execute('''SELECT * FROM Imagens''')
    columns = [desc[0] for desc in cursor.description]
    results = cursor.fetchall()
    df_foto = pd.DataFrame(results, columns=columns)
    
finally:
    # Fechar o cursor e a conexão
    cursor.close()
    conn.close()

In [61]:
display(df_cooperativa)

,id_cooperativa,cnpj,nome,email,senha,telefone
0,1,12345678901243,Cooperativa X,contato@cooperativax.com,senha1234,1198765432
1,2,98425102841331,Cooperativa Y,contato@cooperativay.com,senha5678,2198765432
2,3,81437551000231,Cooperativa C,contato@cooperativac.com,senha6745,3456789012
3,4,10987654322413,Cooperativa Z,contato@cooperativaz.com,senha9123,4567890123
4,5,54637281903419,Cooperativa D,contato@cooperativad.com,senha7346,5678901234
5,6,56473839102934,Cooperativa I,contato@cooperativai.com,senha0157,6789012345


## Se conectando no banco do 2ano

In [62]:
print('Começando===============================================')
try:
    # Conectar ao banco de dados
    conn = psycopg2.connect(
        host=db_host1,
        database=db_database2,
        user=db_user1,
        password=db_password1,
        port=db_port1
    )
    print("Conexão estabelecida com sucesso!")
except psycopg2.OperationalError as e:
    print(f"Erro ao conectar ao banco de dados: {e}")
    exit()  # Saia se a conexão falhar

Começando===============================================
Conexão estabelecida com sucesso!


## Pegando as informações de login da cooperativa

In [63]:
try:
    cursor = conn.cursor()

    cursor.execute('DELETE FROM cooperativa') # deleta todos os registros para garantir que não vai pegar dados duplicados e garante que todos os valores do banco apagados ou alterados seram registrados
                   
    for i in range(len(df_cooperativa)):
        cursor.execute('CALL insert_cooperativa(%s, %s, %s, %s)', 
                       (df_cooperativa['cnpj'][i], df_cooperativa['nome'][i], df_cooperativa['email'][i], df_cooperativa['senha'][i]))

    # Confirme as alterações
    conn.commit()
    print("Deu Green")
except Exception as e:
    print(f"Erro ao executar a procedure: {e}")
    conn.rollback()

Erro ao executar a procedure: Senha menor que 8 dígitos ou não tem numeros ou não possui caracteres especiais!!!
CONTEXT:  PL/pgSQL function insert_cooperativa(character varying,character varying,character varying,character varying) line 9 at RAISE



# Apartir daqui não vamos mais usar, mas esta ai por que talvez seja util.

In [64]:
display(df_produto)
display(df_leilao)
display(df_lance)
display(df_foto)

,id_produto,material,peso,id_leilao
0,1,Plástico,500.0,1
1,2,Metal,300.0,2
2,3,Vidro,200.0,3
3,4,Papel,700.0,4
4,5,Metal,1000.0,5
5,6,Papelao,100.0,6


,id_leilao,data_inicio,data_fim,valor_inicial,detalhes,hora_fim,id_endereco,id_cooperativa
0,1,2024-08-28,2024-09-01,1000.0,"Uma caixa de madeira muito bem, valorizada cri...",12:09:00,1,1
1,2,2024-09-10,2024-09-15,2000.0,Um pacote com cerca de uma tonelada de plastico,13:20:00,2,2
2,3,2024-09-20,None,3000.0,Um pedaço de metal,07:30:20,3,3
3,4,2024-10-25,2024-10-27,4000.0,Varias latinhas de metal,10:08:00,4,4
4,5,2024-05-12,2024-06-02,5000.0,Um rolo de papel enorme,18:09:00,5,5
5,6,2024-11-20,None,6000.0,Varias garrafas de metal enferrujadas,21:20:00,6,6


,id_lance,valor,data_lance,id_leilao,id_empresa
0,1,1200.0,2024-08-29,1,12345678000195
1,2,2500.0,2024-09-11,2,98765432000199
2,3,3200.0,2024-09-21,3,54321678000100
3,4,4200.0,2024-10-22,4,45678912345190
4,5,5200.0,2024-03-11,5,83423741824425
5,6,1100.0,2024-12-21,6,12953649514374


,id_imagem,url,id_produto
0,1,http://example.com/imagem1.jpg,1
1,2,http://example.com/imagem2.jpg,2
2,3,http://example.com/imagem3.jpg,3
3,4,http://example.com/imagem4.jpg,4
4,5,http://example.com/imagem5.jpg,5
5,6,http://example.com/imagem6.jpg,6


In [65]:
# tipo_produto varchar,
#     valor_inicial_produto numeric,
#     valor_final_produto numeric,
#     peso_produto numeric,
#     foto_produto varchar

try:
    cursor = conn.cursor()

    for i in range(len(df_produto)):
        cursor.execute('''INSERT INTO produto (tipo_produto, valor_inicial_produto, peso_produto, foto_produto) 
                       VALUES (%s, %s, %s,%s)''', 
                (df_produto['material'][i],df_leilao['valor_inicial'][i], df_produto['peso'][i],df_foto['url'][i]))

    # Confirme as alterações
    conn.commit()
    print("Deu Green")
except Exception as e:
    print(f"Erro ao executar a procedure: {e}")
    conn.rollback()

Deu Green


In [66]:
display(df_leilao)

,id_leilao,data_inicio,data_fim,valor_inicial,detalhes,hora_fim,id_endereco,id_cooperativa
0,1,2024-08-28,2024-09-01,1000.0,"Uma caixa de madeira muito bem, valorizada cri...",12:09:00,1,1
1,2,2024-09-10,2024-09-15,2000.0,Um pacote com cerca de uma tonelada de plastico,13:20:00,2,2
2,3,2024-09-20,None,3000.0,Um pedaço de metal,07:30:20,3,3
3,4,2024-10-25,2024-10-27,4000.0,Varias latinhas de metal,10:08:00,4,4
4,5,2024-05-12,2024-06-02,5000.0,Um rolo de papel enorme,18:09:00,5,5
5,6,2024-11-20,None,6000.0,Varias garrafas de metal enferrujadas,21:20:00,6,6


In [67]:
# tipo_produto varchar,
#     valor_inicial_produto numeric,
#     valor_final_produto numeric,
#     peso_produto numeric,
#     foto_produto varchar

try:
    cursor = conn.cursor()

    for i in range(len(df_produto)):
        cursor.execute('''INSERT INTO produto (tipo_produto, valor_inicial_produto, peso_produto, foto_produto) 
                       VALUES (%s, %s, %s,%s)''', 
                (df_produto['material'][i],df_leilao['valor_inicial'][i], df_produto['peso'][i],df_foto['url'][i]))

    # Confirme as alterações
    conn.commit()
    print("Deu Green")
except Exception as e:
    print(f"Erro ao executar a procedure: {e}")
    conn.rollback()

Deu Green


In [68]:
try:
    cursor = conn.cursor()

#   (lan_valor numeric, lan_data_lance date, lan_cnpj_empresa varchar, leilao_data_inicio date, leilao_data_fim date, lan_id_leilao int)
    for i in range(len(df_leilao)):
        cursor.execute('CALL insert_leilao(%s::numeric, %s::date, %s::varchar, %s::date, %s::date, %s::int)',
                    (df_leilao['valor_inicial'][i], df_leilao['data_inicio'][i], df_leilao['detalhes'][i], 
                        df_leilao['data_inicio'][i], df_leilao['data_fim'][i], int(df_leilao['id_endereco'][i])))

    # Confirme as alterações
    conn.commit()
    print("Deu Green")
except Exception as e:
    print(f"Erro ao executar a procedure: {e}")
    conn.rollback()
finally:
    # Fechar o cursor e a conexão
    cursor.close()
    conn.close()

Erro ao executar a procedure: Endereço ou produto não existe!!!
CONTEXT:  PL/pgSQL function insert_leilao(numeric,date,character varying,date,date,integer) line 12 at RAISE

